In [42]:
import pyodbc
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori, association_rules

warnings.simplefilter("ignore")

db_path = 'C:/Users/annap/Documentos/codigos/teste_pratico_sicredi/data/compras2014.mdb'
driver = 'Microsoft Access Driver (*.mdb, *.accdb)'

conn = pyodbc.connect(f"Driver={driver};DBQ={db_path};")
cursor = conn.cursor()

In [43]:
def get_tables_db(cursor:pyodbc.Cursor)->list:

    names = []
    for row in cursor.tables():
        if "MSys" not in str(row.table_name):
            names.append(row.table_name)
    
    return names
    
def get_data(table_name:str, conn:pyodbc.connect)->pd.DataFrame:
    
    return pd.read_sql_query("select * from "+table_name, con=conn)

table_names = get_tables_db(cursor)
table_names

['itemtransacao', 'itens', 'transacoes', 'itens Consulta']

In [45]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
categorics = ['category', 'object']

def show_stats_numeric(df:pd.DataFrame):

    columns = df.select_dtypes(include=numerics).columns

    for column in columns:
        print(f"col:{column}\nnulls:{df[column].isnull().sum()}, duplicated:{df[column].duplicated().sum()}, nunique:{df[column].nunique()}, values_uniques:{sorted(df[column].unique())}, value_min:{df[column].min()}, value_max:{df[column].max()} \n")

def show_stats_categorical(df:pd.DataFrame):

    columns = df.select_dtypes(include=categorics).columns

    for column in columns:
        print(f"col:{column}\n{df[column].value_counts()}")

## Itens

In [46]:
df_itens = get_data('itens', conn)
print(f"Shape:{df_itens.shape}\nDuplicated lines:{df_itens.duplicated().sum()}")
df_itens.head()

Shape:(10, 4)
Duplicated lines:0


,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante


In [47]:
show_stats_numeric(df_itens)

col:codItem
nulls:0, duplicated:0, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 



In [48]:
show_stats_categorical(df_itens)

col:descrição
banana prata        1
banana caturra      1
limao Taiti         1
limão siciliano     1
coca                1
coca light          1
coca lemon light    1
omo collors         1
omo progress        1
ariel total         1
Name: descrição, dtype: int64
col:marca
coca      3
banana    2
omo       2
limao     1
limão     1
ariel     1
Name: marca, dtype: int64
col:tipo
fruta           4
sabao em po     3
refrigerante    2
refirgerante    1
Name: tipo, dtype: int64


In [49]:
df_itens[ df_itens['descrição'].str.contains("lim") | df_itens['marca'].str.contains("lim") ]

,codItem,descrição,marca,tipo
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta


In [50]:
df_itens[ df_itens['tipo'].str.contains("ref") ]

,codItem,descrição,marca,tipo
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante


## Transactions

Contém o valor total e o tipo de pagamento.

Representa o volume de vendas feitos: 34


In [51]:
df_transactions = get_data('transacoes', conn)
print(f"Shape:{df_transactions.shape}\nDuplicated lines:{df_transactions.duplicated().sum()}")
df_transactions.head()

Shape:(34, 3)
Duplicated lines:0


,IDTransação,valorTotal,tipo pagamento
0,1,15,ch
1,2,20,ch
2,3,14,es
3,4,19,ch
4,5,15,es


In [52]:
show_stats_numeric(df_transactions)
show_stats_categorical(df_transactions)

col:IDTransação
nulls:0, duplicated:0, nunique:34, values_uniques:[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], value_min:1, value_max:35 

col:valorTotal
nulls:0, duplicated:21, nunique:13, values_uniques:[10, 11, 12, 14, 15, 16, 17, 18, 19, 20, 21, 24, 30], value_min:10, value_max:30 

col:tipo pagamento
es    17
ch    10
cc     7
Name: tipo pagamento, dtype: int64


## Item Transaction

Contém o id da transação e os itens de cada transação.

In [53]:
df_item_transaction = get_data('itemtransacao', conn)
print(f"Shape:{df_item_transaction.shape}\nDuplicated lines:{df_item_transaction.duplicated().sum()}")
df_item_transaction.head()

Shape:(85, 2)
Duplicated lines:0


,IDTransação,item
0,14,1
1,14,3
2,14,6
3,15,1
4,15,3


In [54]:
show_stats_numeric(df_item_transaction)
show_stats_categorical(df_item_transaction)

col:IDTransação
nulls:0, duplicated:58, nunique:27, values_uniques:[1, 2, 3, 5, 6, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35], value_min:1, value_max:35 

col:item
nulls:0, duplicated:75, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 



In [55]:
aux_supp = df_item_transaction.groupby("item")['IDTransação'].count().sort_values(ascending=False).reset_index(name='quantidade_de_saida')
aux_supp['porcentagem'] = ( aux_supp['quantidade_de_saida'] / aux_supp['quantidade_de_saida'].sum() ) * 100
aux_supp

,item,quantidade_de_saida,porcentagem
0,1,12,14.117647
1,6,12,14.117647
2,2,10,11.764706
3,3,9,10.588235
4,4,9,10.588235
5,10,8,9.411765
6,12,8,9.411765
7,7,6,7.058824
8,11,6,7.058824
9,5,5,5.882353


In [56]:
aux_supp.quantile(q=0.5, interpolation='linear', method='single')['porcentagem']

10.0

In [57]:
df_item_transaction.item.value_counts(normalize=True).mean()

0.1

## Consultas

In [58]:
df_consultas = get_data('itens Consulta', conn)
print(f"Shape:{df_consultas.shape}\nDuplicated lines:{df_consultas.duplicated().sum()}")
df_consultas.head()

Shape:(10, 4)
Duplicated lines:0


,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante


In [59]:
show_stats_numeric(df_consultas)
show_stats_categorical(df_consultas)

col:codItem
nulls:0, duplicated:0, nunique:10, values_uniques:[1, 2, 3, 4, 5, 6, 7, 10, 11, 12], value_min:1, value_max:12 

col:descrição
banana prata        1
banana caturra      1
limao Taiti         1
limão siciliano     1
coca                1
coca light          1
coca lemon light    1
omo collors         1
omo progress        1
ariel total         1
Name: descrição, dtype: int64
col:marca
coca      3
banana    2
omo       2
limao     1
limão     1
ariel     1
Name: marca, dtype: int64
col:tipo
fruta           4
sabao em po     3
refrigerante    2
refirgerante    1
Name: tipo, dtype: int64


## Valid Consultas e Itens

In [60]:
df_consultas.codItem.nunique()

10

In [61]:
df_itens.codItem.nunique()

10

In [62]:
df_consultas[ df_consultas.codItem.isin(df_itens.codItem.unique())]

,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po


In [63]:
df_itens[ df_itens.codItem.isin(df_consultas.codItem.unique())]

,codItem,descrição,marca,tipo
0,1,banana prata,banana,fruta
1,2,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante
5,6,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po


In [64]:
valid = pd.merge(df_itens, df_consultas, on='codItem', suffixes=['_it','_con'])

In [65]:
valid

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con
0,1,banana prata,banana,fruta,banana prata,banana,fruta
1,2,banana caturra,banana,fruta,banana caturra,banana,fruta
2,3,limao Taiti,limao,fruta,limao Taiti,limao,fruta
3,4,limão siciliano,limão,fruta,limão siciliano,limão,fruta
4,5,coca,coca,refrigerante,coca,coca,refrigerante
5,6,coca light,coca,refrigerante,coca light,coca,refrigerante
6,7,coca lemon light,coca,refirgerante,coca lemon light,coca,refirgerante
7,10,omo collors,omo,sabao em po,omo collors,omo,sabao em po
8,11,omo progress,omo,sabao em po,omo progress,omo,sabao em po
9,12,ariel total,ariel,sabao em po,ariel total,ariel,sabao em po


In [66]:
valid[valid['descrição_con'] != valid['descrição_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


In [67]:
valid[valid['marca_con'] != valid['marca_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


In [68]:
valid[valid['tipo_con'] != valid['tipo_it']]

,codItem,descrição_it,marca_it,tipo_it,descrição_con,marca_con,tipo_con


## Analise

## Constução da tabela fato

In [70]:
df_fact = pd.merge(df_item_transaction, df_transactions, on='IDTransação')
df_fact = pd.merge(df_fact, df_itens, left_on='item', right_on='codItem')
print(f"Shape:{df_fact.shape}\nDuplicated lines:{df_fact.duplicated().sum()}")
df_fact.head()

Shape:(85, 8)
Duplicated lines:0


,IDTransação,item,valorTotal,tipo pagamento,codItem,descrição,marca,tipo
0,14,1,10,es,1,banana prata,banana,fruta
1,15,1,20,cc,1,banana prata,banana,fruta
2,18,1,12,es,1,banana prata,banana,fruta
3,19,1,20,ch,1,banana prata,banana,fruta
4,22,1,30,es,1,banana prata,banana,fruta


In [71]:
df_fact = df_fact.rename(columns={'item':'idItem', 'IDTransação':'idTransação'})
df_fact = df_fact[['idTransação', 'idItem', 'marca','descrição', 'tipo', 'valorTotal', 'tipo pagamento']]
df_fact = df_fact.sort_values(by=['idTransação', 'idItem'], ascending=True)
df_fact.head()

,idTransação,idItem,marca,descrição,tipo,valorTotal,tipo pagamento
10,1,1,banana,banana prata,fruta,15,ch
65,1,4,limão,limão siciliano,fruta,15,ch
37,1,12,ariel,ariel total,sabao em po,15,ch
38,2,12,ariel,ariel total,sabao em po,20,ch
55,3,2,banana,banana caturra,fruta,14,es


## Functions 

In [72]:
def generate_association_rules(df:pd.DataFrame, min_sup:float, metric_use:str, threshold:float  )->pd.DataFrame:

    frequent_itens = apriori(df, min_support=min_sup, use_colnames=True)

    rules = association_rules(frequent_itens, metric=metric_use, min_threshold=threshold)
    rules = rules.query("lift >= 1").sort_values(by=['support','confidence'], ascending = [False, False])

    return frequent_itens, rules

def format_data_to_apriori(df:pd.DataFrame, columns:list)->pd.DataFrame:

    df_apriori = df[columns]
    df_apriori = pd.get_dummies(df_apriori)
    df_apriori = df_apriori.groupby(columns[0]).sum()
    
    for column in df_apriori.columns:
        df_apriori.loc[ df_apriori[column] > 1 , column] = 1

    return df_apriori

## Criando regras à partir da descrição

In [73]:
df_description = format_data_to_apriori(df_fact, ['idTransação', 'descrição'])
df_description.head()

,descrição_ariel total,descrição_banana caturra,descrição_banana prata,descrição_coca,descrição_coca lemon light,descrição_coca light,descrição_limao Taiti,descrição_limão siciliano,descrição_omo collors,descrição_omo progress
idTransação,,,,,,,,,,
1,1,0,1,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0
3,1,1,0,0,0,0,0,0,0,0
5,0,1,0,1,0,0,0,0,0,0
6,1,0,1,0,0,0,1,0,0,0


In [74]:
frequency_description, rules_description = generate_association_rules(df_description, min_sup=0.1, metric_use='confidence', threshold=0.5)
rules_description.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
6,(descrição_omo progress),(descrição_coca light),0.222222,0.444444,0.185185,0.833333,1.875,0.086420,3.333333
2,(descrição_limão siciliano),(descrição_banana prata),0.333333,0.444444,0.185185,0.555556,1.250,0.037037,1.250000
8,"(descrição_limão siciliano, descrição_coca light)",(descrição_banana caturra),0.148148,0.370370,0.148148,1.000000,2.700,0.093278,inf
9,"(descrição_limão siciliano, descrição_banana c...",(descrição_coca light),0.148148,0.444444,0.148148,1.000000,2.250,0.082305,inf
10,"(descrição_banana caturra, descrição_coca light)",(descrição_limão siciliano),0.148148,0.333333,0.148148,1.000000,3.000,0.098765,inf


In [75]:
rules_description.shape

(18, 9)